---
---

In [1]:
// #include <cassert>
// #include <iostream>

In [1]:
#include <cassert>
#include <iostream>

class Indices {
   // Abstraction for a finite sequence of indices,
   // where each 'index' is a multi-index i.e. it 
   // is itself a fixed-sized sequence of integers,
   // say e.g. ( (0,0), (0,1), (0,2), (1,1), ...).
   // The integers of a current multi-index are
   // held in a buffer. This class serves to modify
   // the buffer to simulate sequence traversal.
   public:
      // By the sequence being exhausted we mean
      // that it has been taken beyond its last element.
      // The flag is not yet set when the indices
      // represent the last element of the sequence.
      virtual bool is_exhausted() = 0;
      //virtual void set_exhausted(bool) = 0;
      // connect some internal pointer to the buffer with indices
      virtual void use_buff(unsigned int* const) = 0;
      // number of indices (lenght of buffer)
      unsigned int k;
      // move the index one step forward in the sequence
      virtual void next() = 0; 
      // change the index back to the first element of the sequence
      virtual void reset() = 0;
      virtual std::string to_str() = 0;
      void print(unsigned limit = -1) {
         assert(index);
         unsigned count = 0;
         while(!is_exhausted()) {
            if (count == limit)
               break;
            std::cout << to_str() << std::endl;
            next();
            ++count;
         }
         reset();
      }
};

class IndicesTriangle : public Indices {
   // Each index in the sequence is an increasing sequence.
   protected:
      // pointer to a buffer with actual integers
      unsigned int* index = NULL;
      // lower bound (inclusive) of values of all indices
      unsigned int lower;
      // upper bound (noninclusive) of values of all indices
      unsigned int upper;
      // is the index-sequence not strictly increasing;
      // i.e. is e.g. (0, 1, 1, 2) in the sequence
      bool with_diag;
      // was the whole sequence exhausted
      bool exhausted = false;
   public:
      // constructor
      IndicesTriangle (unsigned int k_,
                       unsigned int lower_ = 0,
                       unsigned int upper_ = 0,
                       bool with_diag_ = true) :
         lower(lower_),
         upper(upper_),
         with_diag(with_diag_) {
         assert(upper >= lower);
         if( !with_diag & ((upper - lower) < k) ) {
            k = 0;
         }
         else {
            k = k_;
         }
      }
      void use_buff(unsigned int* const buff) {
         index = buff;
      }
      bool is_exhausted() { return exhausted; }
      //void set_exhausted(bool exhausted_) { exhausted = exhausted_; }
      void next() {
         // F-style traversal
         if (k > 0) {
            bool increment = 1;
            for(unsigned int i = 0; i < k - 1; i++) {
               div_t division = div(index[i] + increment, index[i+1] + with_diag);
               increment = division.quot;
               index[i] = division.rem + increment * (lower + !with_diag * i);
            }
            index[k - 1] += increment;
            exhausted = index[k - 1] / upper;
         }
         else {
            exhausted = true;
         }
      }
      void next_() {
         // C-style traversal
         if (k > 0) {
            unsigned int position = k - 1;
            bool shift_left = (index[position] + 1) / upper;
            while (shift_left) {
               if (!position) {
                  exhausted = true;
                  return;
               }
               --position;
               shift_left = (index[position] + !with_diag) / index[position + 1];
            }
            ++index[position];
            for (unsigned int i = position + 1; i < k; ++i) {
               index[i] = index[i - 1] + !with_diag;
            }  
         }
         else {
            exhausted = true;
         }
      }
      void reset() {
         for(unsigned int i = 0; i < k; ++i)
            index[i] = lower + !with_diag * i;
         exhausted = false;
      }
      std::string to_str() {
         std::stringstream ss;
         if (k > 0) {
            ss << "(";
            for (unsigned int i = 0; i < k - 1; ++i)
                ss << index[i] << ", ";
            ss << index[k - 1] << ")";
         }
         else {
            ss << "";
         }
         return ss.str();
      }
};

class IndicesProduct : public Indices {   
   // Cartesian product of two sequences.
   // Again results in a sequence (the order
   // is "row-wise", i.e. second index changes
   // faster).
   protected:
      Indices& ind_L;
      Indices& ind_R;
   public:
      unsigned int boundary_index;
      IndicesProduct(Indices& ind_L_arg, Indices& ind_R_arg) :
         ind_L(ind_L_arg),
         ind_R(ind_R_arg),
         boundary_index(ind_L_arg.k) {
            k = ind_L_arg.k + ind_R_arg.k;
      }
      void use_buff(unsigned int* const buff) {
         ind_L.use_buff(buff);
         ind_R.use_buff(buff + boundary_index);
      }
      bool is_exhausted() {
         return ind_L.is_exhausted();
      }
//       void set_exhausted(bool exhausted_) {
//          ind_L.set_exhausted(exhausted_);
//       }
    
      void next() {
         ind_R.next();
         if (ind_R.is_exhausted()) {
            ind_R.reset();
            ind_L.next();
         }
      }
      void reset() {
         ind_L.reset();
         ind_R.reset();
      }
      std::string to_str() {
         std::stringstream ss;
         ss << ind_L.to_str() << ind_R.to_str();
         return ss.str();
      }
};

class IndicesProductId : public Indices {
   // Multiplication's identity.
   protected:
      bool exhausted = false;
   public:
      IndicesProductId() { k = 0; }
      bool is_exhausted() { return exhausted; }
      //void set_exhausted(bool exhausted_) { exhausted = exhausted_; }
      void use_buff(unsigned int* const buff) {}
      void next() { exhausted = true; }
      void reset() { exhausted = false; }
      std::string to_str() { return ""; }
};

class IndicesSum : public Indices {
   // Concatenation of two sequences.
   // (Assumes that both represent multi-indices
   // of the same length k.)
   // Meaning that, when we step over the sum,
   // we first exhaust the first component sequence,
   // and then step through the second one.
   protected:
      Indices& ind_L;
      Indices& ind_R;
   public:
      bool using_L;
      IndicesSum(Indices &ind_L_arg, Indices &ind_R_arg) :
         ind_L(ind_L_arg),
         ind_R(ind_R_arg),
         using_L(true) {
         assert(ind_L_arg.k == ind_R_arg.k);
         k = ind_L_arg.k;
      }
      void use_buff(unsigned int* const buff) {
         ind_L.use_buff(buff);
         ind_R.use_buff(buff);
//          if (ind_L.is_exhausted()) {
//             using_L = false;
//          }
      }
      bool is_exhausted() {
         return ind_L.is_exhausted() & 
                ind_R.is_exhausted();
      }
//       void set_exhausted(bool exhausted_) {
//          ind_L.set_exhausted(exhausted_);
//          ind_R.set_exhausted(exhausted_);
//       }
      void next() {
         if (using_L) {
            ind_L.next();
            if(ind_L.is_exhausted()) {
               using_L = false;
               ind_R.reset();
            }
         }
         else {
            ind_R.next();
         }
      }
      void reset() {
         using_L = true;
         ind_L.reset();
         if(ind_L.is_exhausted()) {
              using_L = false;
              ind_R.reset();
         }
      }
      std::string to_str() {
         std::stringstream ss;
         ss << (using_L ? ind_L.to_str() : ind_R.to_str());
         return ss.str();
      }
};

class IndicesSumId : public Indices {
   // Summation's identity.
   // Parametrized by k.
//    protected:
//       bool exhausted = true;
   public:
      IndicesSumId(int k_arg) { k = k_arg; }
      bool is_exhausted() { return true; }
      //void set_exhausted(bool exhausted_arg) {}
      void use_buff(unsigned int* const buff) {}
      void next() {}
      void reset() {}
      std::string to_str() { return ""; }
};



In [3]:
const unsigned int k = 3;
IndicesTriangle my_indices_1(k, 2, 6, false);
unsigned int tile_index_1[k];
my_indices_1.use_buff(tile_index_1);
my_indices_1.reset();
my_indices_1.print();

(2, 3, 4)
(2, 3, 5)
(2, 4, 5)
(3, 4, 5)


In [4]:
const unsigned int k_1 = 2;
const unsigned int k_2 = 3; 
IndicesTriangle my_indices_2
                   (k_1, 1, 4, true);
IndicesTriangle my_indices_3
                 (k_2, 6, 10, false);
IndicesProduct my_product_1
        (my_indices_2, my_indices_3);
IndicesProductId my_one
                = IndicesProductId();
IndicesProduct my_product_2
              (my_product_1, my_one);
unsigned int tile_index_2[k_1 + k_2];
my_product_2.use_buff(tile_index_2);
my_product_2.reset();
my_product_2.print();

(1, 1)(6, 7, 8)
(1, 1)(6, 7, 9)
(1, 1)(6, 8, 9)
(1, 1)(7, 8, 9)
(1, 2)(6, 7, 8)
(1, 2)(6, 7, 9)
(1, 2)(6, 8, 9)
(1, 2)(7, 8, 9)
(2, 2)(6, 7, 8)
(2, 2)(6, 7, 9)
(2, 2)(6, 8, 9)
(2, 2)(7, 8, 9)
(1, 3)(6, 7, 8)
(1, 3)(6, 7, 9)
(1, 3)(6, 8, 9)
(1, 3)(7, 8, 9)
(2, 3)(6, 7, 8)
(2, 3)(6, 7, 9)
(2, 3)(6, 8, 9)
(2, 3)(7, 8, 9)
(3, 3)(6, 7, 8)
(3, 3)(6, 7, 9)
(3, 3)(6, 8, 9)
(3, 3)(7, 8, 9)


In [2]:
const unsigned int k = 3;
unsigned int tile_index_1[k];
IndicesTriangle my_indices_4
                  (k, 0, 4, true);
IndicesTriangle my_indices_5
                (k, 6, 10, false);
IndicesSumId my_zero(k);
IndicesSum my_sum_1
          (my_indices_4, my_zero);
IndicesSum my_sum_2
         (my_sum_1, my_indices_5);
my_sum_2.use_buff(tile_index_1);
my_sum_2.reset();
my_sum_2.print()

(0, 0, 0)
(0, 0, 1)
(0, 1, 1)
(1, 1, 1)
(0, 0, 2)
(0, 1, 2)
(1, 1, 2)
(0, 2, 2)
(1, 2, 2)
(2, 2, 2)
(0, 0, 3)
(0, 1, 3)
(1, 1, 3)
(0, 2, 3)
(1, 2, 3)
(2, 2, 3)
(0, 3, 3)
(1, 3, 3)
(2, 3, 3)
(3, 3, 3)
(6, 7, 8)
(6, 7, 9)
(6, 8, 9)
(7, 8, 9)


In [5]:
// IndicesSum my_sum(my_indices_1, my_indices_2);
// my_sum.use_buff(tile_index);

In [6]:
// my_sum.print()

In [2]:
IndicesSum& fold(std::vector<IndicesTriangle>& vect_1,
                 std::vector<IndicesTriangle>& vect_2,
                 std::vector<IndicesProduct>& vect_1_x_2,
                 std::vector<IndicesSum>& vect_1_x_2_sum,
                 IndicesSumId& indices_zero,
                 unsigned int index_1_L,
                 unsigned int index_1_R,
                 unsigned int index_2_L,
                 unsigned int index_2_R,
                 unsigned int lower_limit,
                 unsigned int upper_limit) {
   // construct
   // indices =
   // \sum_{i = lower_limit}^{upper_limit}
   //   [ (index_1_L : index_1_R)^(upper_limit - i)
   //     \times
   //     (index_2_L : index_2_R)^i
   //   ],
   // where (L : R)^i denotes IndicesTriangle(i, L, R, true)
    
   assert(lower_limit <= upper_limit);
   unsigned int component_count = upper_limit - lower_limit + 1;
   vect_1.reserve(component_count);
   vect_2.reserve(component_count);
   vect_1_x_2.reserve(component_count);
   vect_1_x_2_sum.reserve(component_count);

   for(unsigned int i = lower_limit; i <= upper_limit; ++i) {
      vect_1.emplace_back(upper_limit - i,
                          index_1_L,
                          index_1_R,
                          true);
      vect_2.emplace_back(i,
                          index_2_L,
                          index_2_R,
                          true);
      vect_1_x_2.emplace_back(vect_1.back(),
                              vect_2.back());
   }
   vect_1_x_2_sum.emplace_back(indices_zero, vect_1_x_2[0]);
   for(unsigned int i = 1; i < component_count; ++i) {
      vect_1_x_2_sum.emplace_back(
            vect_1_x_2_sum.back(),
            vect_1_x_2[i]);
   }
   return vect_1_x_2_sum.back();
}


$$
\sum_{i = 1}^{k=3} (0 : 3)^{k - i} \times (3 : 5)^{i}
$$

In [3]:
const unsigned int k = 3;
std::vector<IndicesTriangle> vect_1;
std::vector<IndicesTriangle> vect_2;
std::vector<IndicesProduct> vect_1_x_2;
std::vector<IndicesSum> vect_1_x_2_sum;
IndicesSumId indices_zero(k);
unsigned int tile_index[k];
IndicesSum& fold_result =
    fold(vect_1, vect_2, vect_1_x_2,
         vect_1_x_2_sum, indices_zero,
         0, 3, 3, 5, 1, k);
fold_result.use_buff(tile_index);
fold_result.reset();
fold_result.print();

(0, 0)(3)
(0, 0)(4)
(0, 1)(3)
(0, 1)(4)
(1, 1)(3)
(1, 1)(4)
(0, 2)(3)
(0, 2)(4)
(1, 2)(3)
(1, 2)(4)
(2, 2)(3)
(2, 2)(4)
(0)(3, 3)
(0)(3, 4)
(0)(4, 4)
(1)(3, 3)
(1)(3, 4)
(1)(4, 4)
(2)(3, 3)
(2)(3, 4)
(2)(4, 4)
(3, 3, 3)
(3, 3, 4)
(3, 4, 4)
(4, 4, 4)


In [3]:
template<unsigned int dim>
IndicesSum& build_gpu_tile_indices(unsigned int groups[]) {
    static IndicesTriangle reg_fat(dim, groups[0], groups[1], true);
    static IndicesTriangle reg_fat_(dim - 1, groups[0], groups[1], true);
    static IndicesTriangle heur_fat(1, groups[2], groups[3], true);
    static IndicesTriangle contr_fat(1, groups[4], groups[5], true);
    static IndicesSum heur_plus_contr_fat(heur_fat, contr_fat);
    static IndicesProduct fat_(reg_fat_, heur_plus_contr_fat);
    static IndicesSum gpu_tile_indices(reg_fat, fat_);
    return gpu_tile_indices;
}

In [4]:
unsigned int groups[7] =
            {0,2,6,11,15,17,20};
const unsigned int dim = 3;
IndicesSum& gpu_tile_indices =
    build_gpu_tile_indices<dim>(groups);
unsigned int buff[dim];
gpu_tile_indices.use_buff(buff);
gpu_tile_indices.reset();
gpu_tile_indices.print();

(0, 0, 0)
(0, 0, 1)
(0, 1, 1)
(1, 1, 1)
(0, 0)(6)
(0, 0)(7)
(0, 0)(8)
(0, 0)(9)
(0, 0)(10)
(0, 0)(15)
(0, 0)(16)
(0, 1)(6)
(0, 1)(7)
(0, 1)(8)
(0, 1)(9)
(0, 1)(10)
(0, 1)(15)
(0, 1)(16)
(1, 1)(6)
(1, 1)(7)
(1, 1)(8)
(1, 1)(9)
(1, 1)(10)
(1, 1)(15)
(1, 1)(16)


In [5]:
template<unsigned int dim>
IndicesSum& build_cpu_tile_indices(unsigned int groups[]) {
    // \sum_{i=1}^{dim} (0:1)^{dim-i} (1:2)^i
    static std::vector<IndicesTriangle> reg_fat;
    static std::vector<IndicesTriangle> reg_lean;
    static std::vector<IndicesProduct> reg_fat_x_lean;
    static std::vector<IndicesSum> reg_fat_x_lean_sum;
    static IndicesSumId indices_zero(dim);
    IndicesSum& rectangle_1 = fold(reg_fat, reg_lean,
                                   reg_fat_x_lean,
                                   reg_fat_x_lean_sum,
                                   indices_zero,
                                   groups[0], groups[1],
                                   groups[1], groups[2],
                                   1, dim);
    // ( \sum_{i=1}^{dim} (0:1)^{dim-1-i} (1:2)^i ) (2:6)
    static std::vector<IndicesTriangle> reg_fat_;
    static std::vector<IndicesTriangle> reg_lean_;
    static std::vector<IndicesProduct> reg_fat_x_lean_;
    static std::vector<IndicesSum> reg_fat_x_lean_sum_;
    static IndicesSumId indices_zero_(dim - 1);
    IndicesSum& reg_fold = fold(reg_fat_, reg_lean_,
                                reg_fat_x_lean_,
                                reg_fat_x_lean_sum_,
                                indices_zero_,
                                groups[0], groups[1],
                                groups[1], groups[2],
                                1, dim - 1);
    static IndicesTriangle non_reg(1, groups[2], groups[6], true);
    static IndicesProduct rectangle_2(reg_fold, non_reg);
    // (0:1)^{dim-1} (3:4 + 5:6)
    static IndicesTriangle reg_fat__(dim - 1, groups[0], groups[1, true]);
    static IndicesTriangle heur_lean(1, groups[3], groups[4], true);
    static IndicesTriangle contr_lean(1, groups[5], groups[6], true);
    static IndicesSum non_reg_lean(heur_lean, contr_lean);
    static IndicesProduct rectangle_3(reg_fat[0], non_reg_lean);
    // 
    static IndicesSum rectangle_1_plus_2(rectangle_1, rectangle_2);
    static IndicesSum rectangle(rectangle_1_plus_2, rectangle_3);
    return rectangle;
}

In [6]:
unsigned int groups[7] =
            {0,2,6,11,15,17,20};
const unsigned int dim = 3;
unsigned int buff[dim];

IndicesSum& cpu_tile_indices =
    build_cpu_tile_indices<dim>(groups);
cpu_tile_indices.use_buff(buff);
cpu_tile_indices.reset();
cpu_tile_indices.print()

(0, 0)(2)
(0, 0)(3)
(0, 0)(4)
(0, 0)(5)
(0, 1)(2)
(0, 1)(3)
(0, 1)(4)
(0, 1)(5)
(1, 1)(2)
(1, 1)(3)
(1, 1)(4)
(1, 1)(5)
(0)(2, 2)
(0)(2, 3)
(0)(3, 3)
(0)(2, 4)
(0)(3, 4)
(0)(4, 4)
(0)(2, 5)
(0)(3, 5)
(0)(4, 5)
(0)(5, 5)
(1)(2, 2)
(1)(2, 3)
(1)(3, 3)
(1)(2, 4)
(1)(3, 4)
(1)(4, 4)
(1)(2, 5)
(1)(3, 5)
(1)(4, 5)
(1)(5, 5)
(2, 2, 2)
(2, 2, 3)
(2, 3, 3)
(3, 3, 3)
(2, 2, 4)
(2, 3, 4)
(3, 3, 4)
(2, 4, 4)
(3, 4, 4)
(4, 4, 4)
(2, 2, 5)
(2, 3, 5)
(3, 3, 5)
(2, 4, 5)
(3, 4, 5)
(4, 4, 5)
(2, 5, 5)
(3, 5, 5)
(4, 5, 5)
(5, 5, 5)
(0)(2)(6)
(0)(2)(7)
(0)(2)(8)
(0)(2)(9)
(0)(2)(10)
(0)(2)(11)
(0)(2)(12)
(0)(2)(13)
(0)(2)(14)
(0)(2)(15)
(0)(2)(16)
(0)(2)(17)
(0)(2)(18)
(0)(2)(19)
(0)(3)(6)
(0)(3)(7)
(0)(3)(8)
(0)(3)(9)
(0)(3)(10)
(0)(3)(11)
(0)(3)(12)
(0)(3)(13)
(0)(3)(14)
(0)(3)(15)
(0)(3)(16)
(0)(3)(17)
(0)(3)(18)
(0)(3)(19)
(0)(4)(6)
(0)(4)(7)
(0)(4)(8)
(0)(4)(9)
(0)(4)(10)
(0)(4)(11)
(0)(4)(12)
(0)(4)(13)
(0)(4)(14)
(0)(4)(15)
(0)(4)(16)
(0)(4)(17)
(0)(4)(18)
(0)(4)(19)
(0)(5)(6)
(0)(5)(7)
(0)(5)(8)


---
---

In [13]:
#include <algorithm>
#include <cstdint>
#include <cstring>
#include <random>
#include <vector>

void discretize(
    uint32_t seed,
    uint32_t discretization_index,
    //uint32_t feature_id,
    unsigned int feature_id,
    std::size_t divisions,
    std::size_t object_count,
    const double* in_data,
    const std::vector<double>& sorted_in_data,
    //uint8_t* out_data,
    unsigned int* out_data,
    double range
) {
    double* thresholds = new double[divisions];

    // brackets to limit scope
    {
        double sum = 0.0f;
        // brackets to limit scope
        {
            std::mt19937 seed_random_generator0(seed);
            std::mt19937 seed_random_generator1(seed_random_generator0() ^ discretization_index);
            std::mt19937 random_generator(seed_random_generator1() ^ feature_id);

            // E(X) = (a + b) / 2 = (1 - range + 1 + range) / 2 = 1
            std::uniform_real_distribution<double> uniform_range(1.0f - range, 1.0f + range);

            for (std::size_t d = 0; d < divisions; ++d) {
                thresholds[d] = uniform_range(random_generator);
                sum += thresholds[d];
            }

            sum += uniform_range(random_generator);
        }

        std::size_t done = 0;
        const double length_step = static_cast<double>(object_count) / sum;

        // thresholds are converted from an arbitrary space into real values (via indices)
        // d - iterates over divisions (of a variable in a discretization)
        for (std::size_t d = 0; d < divisions; ++d) {
            done += std::lround(thresholds[d] * length_step);

//             // Note: Check when will this happen, maybe could be skipped
//             if (done >= object_count) {
//                 done = object_count - 1;
//             }

            thresholds[d] = sorted_in_data[done];
        }
    }

    // o - iterates over objects
    for (std::size_t o = 0; o < object_count; ++o) {
        out_data[o] = 0;

        // out_data[o] (starting with 0) is incremented every time in_data[o] is above a threshold
        // divisions is a small number (<=15), no reason to use binsearch, hence linear
        // d - iterates over divisions (per object o)
        for (std::size_t d = 0; d < divisions; ++d) {
            out_data[o] += in_data[o] > thresholds[d];
        }
    }

    delete[] thresholds;
}

In [14]:
std::mt19937 random_generator(0);
std::uniform_real_distribution<double> uniform_range(0, 1);
double example_in[10];
unsigned int example_out[10];
for (int i = 0; i < 10; ++i)
    example_in[i] = uniform_range(random_generator);
std::vector<double> example_in_vec(example_in, example_in + 10);
std::sort(example_in_vec.begin(), example_in_vec.end());
discretize(0, 0, 0, 2, 10, example_in, example_in_vec, example_out, 0);
example_in

{ 0.592845, 0.844266, 0.857946, 0.847252, 0.623564, 0.384382, 0.297535, 0.056713, 0.272656, 0.477665 }

In [15]:
example_out

{ 1, 2, 2, 2, 1, 0, 0, 0, 0, 1 }

---
---

In [1]:
#include <iostream>
std::cout.precision(10);
#include <math.h>
#include <stdlib.h>
#include <string>
#include <fstream>


template<unsigned int dim, typename T>
struct Matrix {
    // pointer to a dynamic array,
    // size and shape information
    // and methods to use multidimensional indices
    unsigned int size = 1;
    unsigned int shape[dim];
    T* data = NULL;
    ~Matrix() { delete[] data; }
    void allocate(unsigned int (&shape_)[dim]);
    void allocate(unsigned int (&&shape_)[dim]) { allocate(shape_); }
    void fill_with(T value);
    T& elem(unsigned int (&index)[dim]);
    T& elem(unsigned int (&&index)[dim]) { return elem(index); }
    void increment_multidim_index(unsigned int (&multidim_index)[dim]);
    void print(); // cout size, shape and data
};

template<unsigned int dim, typename T>
void Matrix<dim, T>::allocate(unsigned int (&shape_)[dim]) {
    // mutates size
    if(data) { delete[] data; }
    for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
        shape[dim_index] = shape_[dim_index];
        size *= shape_[dim_index];
    }
    data = new T[size];
}

template<unsigned int dim, typename T>
void Matrix<dim, T>::fill_with(T initial_value) {
    for(unsigned int flat_index = 0; flat_index < size; ++flat_index)
        data[flat_index] = initial_value;
}

template<unsigned int dim, typename T>
T& Matrix<dim, T>::elem(unsigned int (&multidim_index)[dim]) {
    unsigned int offset = 0;
    unsigned int stride = 1;
    for(int dim_index = dim - 1; dim_index >= 0; --dim_index) {
        offset += stride * multidim_index[dim_index];
        stride *= shape[dim_index];
    }
    return data[offset];
}

template<unsigned int dim, typename T>
void Matrix<dim, T>::increment_multidim_index(unsigned int (&multidim_index)[dim]) {
    bool increment;
    for(int dim_index = dim - 1; dim_index >= 0; --dim_index) {
        ++multidim_index[dim_index];
        increment = multidim_index[dim_index] / shape[dim_index];
        if(!increment) { break; }
        multidim_index[dim_index] = 0;
    }
}

template<unsigned int dim, typename T>
void Matrix<dim, T>::print() {
    std::cout << "size: " << size << std::endl;
    std::cout << "shape: {";
    for(unsigned int dim_index = 0; dim_index < dim - 1; ++dim_index)
        std::cout << shape[dim_index] << ", ";
    std::cout << shape[dim - 1] << "}" << std::endl;
    if(data) {
        std::cout << "elements:" << std::endl;
        unsigned int multidim_index[dim] = {0};
        for(unsigned int flat_index = 0; flat_index < size;
            increment_multidim_index(multidim_index), ++flat_index){
            std::cout << flat_index << " [";
            for(unsigned int dim_index = 0; dim_index < dim - 1; ++dim_index)
                std::cout << multidim_index[dim_index] << ", ";
            std::cout << multidim_index[dim - 1] << "] " << data[flat_index] << std::endl;
        }
        std::cout << std::endl;
    }
}


template<unsigned int dim>
struct Contingency : Matrix<dim, double> {
    // matrix class with a method to compute the IGs
    void allocate_margins(Contingency<dim - 1> (&margins)[dim]);
    void compute_margins(Contingency<dim - 1> (&margins)[dim]);
    double linlog();
    void compute_IGs(double (&IGs)[dim - 1],
                     Contingency<dim - 1> (&margins_1)[dim],
                     Contingency<dim - 2> (&margins_2)[dim - 1]);
};

template<unsigned int size>
void copy_skipping(unsigned int (&from)[size],
                   unsigned int (&to)[size - 1],
                   unsigned int skip_index) {
    bool skipped = false;
    for(unsigned int index = 0; index < size; ++index) {
        if(index == skip_index) { skipped = true; continue; }
        to[index - skipped] = from[index];
    }
}

template<unsigned int dim>
void Contingency<dim>::allocate_margins(Contingency<dim - 1> (&margins)[dim]) {
    unsigned int margin_shape[dim - 1];
    for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
        copy_skipping(this->shape, margin_shape, dim_index);
        margins[dim_index].allocate(margin_shape);
    }
}
 
template<unsigned int dim>
void Contingency<dim>::compute_margins(Contingency<dim - 1> (&margins)[dim]) {
    unsigned int multidim_index[dim] = {0};
    unsigned int in_margin_index[dim - 1];
    for(unsigned int flat_index = 0; flat_index < this->size;
        this->increment_multidim_index(multidim_index), ++flat_index) {
        for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
            copy_skipping(multidim_index, in_margin_index, dim_index);
            margins[dim_index].elem(in_margin_index) += this->data[flat_index];
        }
    }  
}

template<unsigned int dim>
double Contingency<dim>::linlog() {
    double result = 0.;
    for(int flat_index = 0; flat_index < this->size; ++flat_index)
        result += this->data[flat_index] * log2(this->data[flat_index]);
    return result;
}

// template<unsigned int dim>
// void Contingency<dim>::compute_IGs(double (&IGs)[dim - 1]) {
//     Contingency<dim - 1> margins_1[dim];
//     Contingency<dim - 2> margins_2[dim - 1];
//     Contingency<dim>::compute_IGs(IGs, margins_1, margins_2);
// }

template<unsigned int dim>
void Contingency<dim>::compute_IGs(double (&IGs)[dim - 1],
                                   Contingency<dim - 1> (&margins_1)[dim],
                                   Contingency<dim - 2> (&margins_2)[dim - 1]) {
    double linlog_global = linlog();
    compute_margins(margins_1);
    double linlog_y = margins_1[dim - 1].linlog();
    margins_1[dim - 1].compute_margins(margins_2);
    for(unsigned int dim_index = 0; dim_index < dim - 1; ++dim_index) {
        double linlog_X = margins_1[dim_index].linlog();
        double linlog_yX = margins_2[dim_index].linlog();
        IGs[dim_index] += linlog_global - linlog_X - linlog_y + linlog_yX;
    }
}

In [2]:
const unsigned int dim = 3;
Contingency<dim> my_contingency;
unsigned int shape[dim] = {2, 4, 3};
my_contingency.allocate(shape);
my_contingency.fill_with(1e-3);
for(unsigned int i = 0; i < my_contingency.size; ++i)
    my_contingency.data[i] += rand() % 10;
my_contingency.print()

size: 24
shape: {2, 4, 3}
elements:
0 [0, 0, 0] 2.001
1 [0, 0, 1] 2.001
2 [0, 0, 2] 5.001
3 [0, 1, 0] 3.001
4 [0, 1, 1] 4.001
5 [0, 1, 2] 4.001
6 [0, 2, 0] 7.001
7 [0, 2, 1] 3.001
8 [0, 2, 2] 0.001
9 [0, 3, 0] 5.001
10 [0, 3, 1] 0.001
11 [0, 3, 2] 3.001
12 [1, 0, 0] 8.001
13 [1, 0, 1] 8.001
14 [1, 0, 2] 6.001
15 [1, 1, 0] 7.001
16 [1, 1, 1] 6.001
17 [1, 1, 2] 7.001
18 [1, 2, 0] 6.001
19 [1, 2, 1] 6.001
20 [1, 2, 2] 7.001
21 [1, 3, 0] 9.001
22 [1, 3, 1] 8.001
23 [1, 3, 2] 7.001



In [3]:
double IGs[dim - 1];
Contingency<dim - 1> margins_1[dim];
Contingency<dim - 2> margins_2[dim - 1];
my_contingency.allocate_margins(margins_1);
margins_1[dim - 1].allocate_margins(margins_2);
for(unsigned int dim_index = 0; dim_index < dim + 1; ++dim_index) {
    margins_1[dim_index].fill_with(0.);
}
for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
    margins_2[dim_index].fill_with(0.);
}
my_contingency.compute_IGs(IGs, margins_1, margins_2);
IGs

{ 11.4359, 12.1768 }

In [19]:
// IGs

---

In [20]:
!ls

check_dummy.ipynb
cpp_playground.ipynb
create_dummy_csv.ipynb
dummy_discretized.csv
dummy_range_groups.csv
dummy_tile_bounds.csv
dummy_X.csv
dummy_y.csv
failed_openmp.ipynb
my_sandbox.ipynb


In [21]:
#include <algorithm>

struct Data2DFromFile : Matrix<2, double> {
    const std::string file_name;
    const char separator;
    Data2DFromFile(std::string &&file_name_,
                   char &&separator_):
        file_name(file_name_),
        separator(separator_) {
    }
    void allocate() {
        // check dimensions of the csv data
        // and use it to initialize the matrix
        unsigned int row_count = 0;
        unsigned int column_count = 0;
        std::ifstream input_file(file_name);
        std::string text_line, text_placeholder;
        std::getline(input_file, text_line); ++row_count;
        std::stringstream text_nums(text_line);
        while(std::getline(text_nums, text_placeholder, separator))
            ++column_count;
        while(std::getline(input_file, text_line))
            ++row_count;
        input_file.close();
        Matrix<2, double>::allocate({column_count, row_count});
    }
    void read_data() {
        // populate the matrix with data from the file
        if(!this->data) { throw; }
        std::ifstream input_file(file_name);
        std::string text_line, text_num;
        for(unsigned int j = 0; j < this->shape[1]; ++j) {
            std::getline(input_file, text_line);
            std::stringstream text_nums(text_line);
            for(unsigned i = 0; i < this->shape[0]; ++i) {
                std::getline(text_nums, text_num, separator);
                this->elem({i, j}) = std::stod(text_num);
            }
        }
        input_file.close();
    }
    void print() {
        std::cout << "data from: " << file_name << std::endl;
        std::cout << "sep: " << separator << std::endl;
        Matrix<2, double>::print();
    }
};


struct NumbersFromFile {
    const std::string file_name;
    unsigned int* data = NULL;
    unsigned int size = 0;
    NumbersFromFile(std::string &file_name_):
        file_name(file_name_) {
    }
    NumbersFromFile(std::string &&file_name_) {
        return NumbersFromFile(file_name_);
    }
    ~NumbersFromFile() { delete data; }
    void initialize() {
        if(data) { throw; }
        std::ifstream input_file(file_name);
        std::string text_line;
        while(std::getline(input_file, text_line))
            ++size;
        input_file.close();
        data = new unsigned int[size];
        for(unsigned i = 0; i < size; ++i) {
            data[i] = 0;
        }
    }
    void read_data() {
        if(!data) { throw; }
        std::ifstream input_file(file_name);
        std::string text_line;
        for(unsigned i = 0; i < size; ++i) {
            std::getline(input_file, text_line);
            data[i] = static_cast<unsigned int>(std::stoi(text_line));
        }
        input_file.close();
    }
    void print() {
        std::cout << "data from: " << file_name << std::endl;
        if(data) {
            std::cout << "elements: " << std::endl;
            for(int i = 0; i < size - 1; ++i)
                std::cout << data[i] << " ";
            std::cout << data[size - 1] << std::endl;
        }
    }
};

struct DataY : NumbersFromFile {
    unsigned int dof = 0;
    DataY(std::string &&file_name_):
        NumbersFromFile{ file_name_ } {
    }
    void read_data() {
        NumbersFromFile::read_data();
        dof = *std::max_element(this->data, this->data + this->size) + 1;
    }
    void print() {
        std::cout << "dof: " << dof << std::endl;
        NumbersFromFile::print();
    }
};

struct ColumnRangeInfo {
    NumbersFromFile bounds;
    NumbersFromFile discretized;
    NumbersFromFile groups;
    unsigned int range_count = 0;  // number of column-ranges
    unsigned int max_range_width = 0;
    unsigned int* dofs = NULL;
    ColumnRangeInfo(std::string &&file_name_bounds,
                    std::string &&file_name_discretized,
                    std::string &&file_name_groups,
                    unsigned int dof) :
        bounds(file_name_bounds),
        discretized(file_name_discretized),
        groups(file_name_groups) {
        bounds.initialize(); bounds.read_data();
        discretized.initialize(); discretized.read_data();
        groups.initialize(); groups.read_data();
        range_count = bounds.size - 1;
        if(range_count != discretized.size) { throw; }
        for(unsigned int range_index = 0; range_index < range_count; ++range_index)
            max_range_width = std::max(max_range_width, bounds.data[range_index + 1] - bounds.data[range_index]);
        dofs = new unsigned int[range_count];
        for(unsigned int range_index = 0; range_index < range_count; ++range_index)
            dofs[range_index] = dof;
    }
    ~ColumnRangeInfo() { delete dofs; }
    void compute_dofs(Data2DFromFile& Xt) {
        for(unsigned int range_index = 0; range_index < range_count; ++range_index) {
            if((bounds.data[range_index] == bounds.data[range_index + 1])  // empty range
               | !discretized.data[range_index]) { continue; }             // a to-be-discretized column
            unsigned int flat_index_0, flat_index_1, dof;
            flat_index_0 = Xt.shape[1] * bounds.data[range_index];
            flat_index_1 = Xt.shape[1] * bounds.data[range_index + 1];
            dof = static_cast<unsigned int>
                (*std::max_element(Xt.data + flat_index_0, Xt.data + flat_index_1)) + 1;
            dofs[range_index] = dof;
        }
    }
};

In [22]:
Data2DFromFile Xt("dummy_X.csv", '\t');
Xt.allocate();
Xt.read_data();

In [23]:
// Xt.print()

In [24]:
// Xt.elem({0, 2})

In [25]:
DataY y("dummy_y.csv");
y.initialize();
y.read_data();

In [26]:
// y.print()

In [27]:
ColumnRangeInfo column_range_info("dummy_tile_bounds.csv", "dummy_discretized.csv", "dummy_range_groups.csv", 5);
column_range_info.compute_dofs(Xt);

In [28]:
column_range_info.max_range_width

124

In [29]:
column_range_info.groups.print()

data from: dummy_range_groups.csv
elements: 
0 2 6 11 15 17 20


In [30]:
column_range_info.bounds.print()

data from: dummy_tile_bounds.csv
elements: 
0 100 200 222 235 247 258 358 458 558 658 758 800 842 876 876 976 1076 1200 1200 1245


In [31]:
for(int range_index = 0; range_index < column_range_info.range_count; ++range_index){
    std::cout << column_range_info.dofs[range_index] << std::endl;
}

5
5
5
6
7
8
5
5
5
5
5
5
6
7
5
5
5
6
5
8


In [32]:
//dim

In [33]:
const unsigned int dim = 3;
unsigned int cpu_tile_index[dim];

IndicesSum& cpu_tile_indices =
    build_cpu_tile_indices<dim>(column_range_info.groups.data);
cpu_tile_indices.use_buff(cpu_tile_index);
cpu_tile_indices.print()


(0, 0)(2)
(0, 0)(3)
(0, 0)(4)
(0, 0)(5)
(0, 1)(2)
(0, 1)(3)
(0, 1)(4)
(0, 1)(5)
(1, 1)(2)
(1, 1)(3)
(1, 1)(4)
(1, 1)(5)
(0)(2, 2)
(0)(2, 3)
(0)(3, 3)
(0)(2, 4)
(0)(3, 4)
(0)(4, 4)
(0)(2, 5)
(0)(3, 5)
(0)(4, 5)
(0)(5, 5)
(1)(2, 2)
(1)(2, 3)
(1)(3, 3)
(1)(2, 4)
(1)(3, 4)
(1)(4, 4)
(1)(2, 5)
(1)(3, 5)
(1)(4, 5)
(1)(5, 5)
(2, 2, 2)
(2, 2, 3)
(2, 3, 3)
(3, 3, 3)
(2, 2, 4)
(2, 3, 4)
(3, 3, 4)
(2, 4, 4)
(3, 4, 4)
(4, 4, 4)
(2, 2, 5)
(2, 3, 5)
(3, 3, 5)
(2, 4, 5)
(3, 4, 5)
(4, 4, 5)
(2, 5, 5)
(3, 5, 5)
(4, 5, 5)
(5, 5, 5)
(0)(2)(6)
(0)(2)(7)
(0)(2)(8)
(0)(2)(9)
(0)(2)(10)
(0)(2)(11)
(0)(2)(12)
(0)(2)(13)
(0)(2)(14)
(0)(2)(15)
(0)(2)(16)
(0)(2)(17)
(0)(2)(18)
(0)(2)(19)
(0)(3)(6)
(0)(3)(7)
(0)(3)(8)
(0)(3)(9)
(0)(3)(10)
(0)(3)(11)
(0)(3)(12)
(0)(3)(13)
(0)(3)(14)
(0)(3)(15)
(0)(3)(16)
(0)(3)(17)
(0)(3)(18)
(0)(3)(19)
(0)(4)(6)
(0)(4)(7)
(0)(4)(8)
(0)(4)(9)
(0)(4)(10)
(0)(4)(11)
(0)(4)(12)
(0)(4)(13)
(0)(4)(14)
(0)(4)(15)
(0)(4)(16)
(0)(4)(17)
(0)(4)(18)
(0)(4)(19)
(0)(5)(6)
(0)(5)(7)
(0)(5)(8)

In [34]:
// IndicesSum& gpu_tile_indices =
//     build_gpu_tile_indices<dim>(column_range_info.groups.data);
// gpu_tile_indices.use_buff(buff);
// gpu_tile_indices.print()

In [35]:
// #include<map>
// std::pair<int,int>(2,3)
// std::map<std::string, int> s2i;
// s2i["haha"] = 3;
// s2i

In [36]:
template<unsigned int dim>
struct TileData : Matrix<2, unsigned int> {
    // the class stores data in a single tile,
    // i.e. cartesian product of dim column-ranges
    // which are specified by a tile_index
    unsigned int corner_columns_0[dim];  // N-W corner of the tile
    unsigned int corner_columns_1[dim];  // S-E corner of the tile
    unsigned int range_dofs[dim + 1];    // dof per dimension
    unsigned int tile_dof;               // product of dofs
    ColumnRangeInfo& column_range_info;
    Data2DFromFile& Xt;
    DataY& y;
    TileData(ColumnRangeInfo& column_range_info_,
             Data2DFromFile& Xt_,
             DataY& y_) :
        column_range_info(column_range_info_),
        Xt(Xt_),
        y(y_) {
        this->allocate({column_range_info.max_range_width * dim, Xt.shape[1]});
    };
    void get_data(unsigned int (&&tile_index)[dim]) {
        get_data(tile_index);
    }
    void get_data(unsigned int (&tile_index)[dim]) {
        range_dofs[dim] = y.dof;
        tile_dof = 1;
        unsigned int discretized, dof, column_index_0, column_index_1,
                     flat_offset_0, flat_offset_1, flat_offset, local_flat_offset;
        // loop over dimensions of the tile
        for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
            // are columns in the range discretized?
            discretized = column_range_info.discretized.data[tile_index[dim_index]];
            // how many dof do those column have?
            dof = column_range_info.dofs[tile_index[dim_index]];
            range_dofs[dim_index] = dof;
            tile_dof *= dof;
            // get actual column indices at left and right borders of the range
            column_index_0 = column_range_info.bounds.data[tile_index[dim_index]];
            corner_columns_0[dim_index] = column_index_0;
            column_index_1 = column_range_info.bounds.data[tile_index[dim_index] + 1];
            corner_columns_1[dim_index] = column_index_1;
            // translate the column indices into flat indices in the C-arrays
            flat_offset_0 = Xt.shape[1] * column_index_0;
            flat_offset_1 = Xt.shape[1] * column_index_1;
            local_flat_offset = dim_index * Xt.shape[1] * column_range_info.max_range_width;
            // move data from the external Data2DFromFile to this object
            if (discretized) {
                // just copy a bunch of columns casting to uint
                for(; flat_offset_0 < flat_offset_1; ++flat_offset_0, ++local_flat_offset) {
                    this->data[local_flat_offset] = static_cast<unsigned int>(Xt.data[flat_offset_0]);
                }
            }
            else {
                // discretize column by column
                for(; flat_offset_0 < flat_offset_1;
                    flat_offset_0 += Xt.shape[1], local_flat_offset += Xt.shape[1]) {
                    std::vector<double> column_vec(Xt.data + flat_offset_0, Xt.data + flat_offset_0 + Xt.shape[1]);
                    std::sort(column_vec.begin(), column_vec.end());
                    discretize(0, 0, flat_offset_0, dof - 1, Xt.shape[1],
                               Xt.data + flat_offset_0, column_vec, this->data + local_flat_offset, 0.);
                }
            }
        }
    }
//     void fill_in_contingency(Contingency<dim + 1>& contingency,
//                              unsigned int (&&column_indices)[dim]) {
//         fill_in_contingency(contingency, column_indices);
//     }
    void fill_in_contingency(Contingency<dim + 1>& contingency,
                             unsigned int (&column_indices)[dim]) {
        unsigned int intile_column_indices[dim];
        for (unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
            assert(column_indices[dim] >= corner_columns_0[dim_index]);
            assert(column_indices[dim] < corner_columns_1[dim_index]);
            intile_column_indices[dim_index] = column_indices[dim_index] - corner_columns_0[dim_index];
        }
        unsigned int multidim_index[dim + 1];
        unsigned int flat_column_index;
        for(unsigned int row_index = 0; row_index < this->shape[1]; ++row_index) {
            for (unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
                flat_column_index = intile_column_indices[dim_index] + dim_index * column_range_info.max_range_width;
                multidim_index[dim_index] = this->elem({flat_column_index, row_index});
            }
            multidim_index[dim] = y.data[row_index];
//             for(int i = 0; i < dim + 1; ++i)
//                 std::cout << multidim_index[i] << " ";
//             std::cout << std::endl;
            ++contingency.elem(multidim_index);
        }
    }
};

In [37]:
TileData<dim> tile_data(column_range_info, Xt, y);

In [38]:
dim

3

In [39]:
cpu_tile_index

{ 0, 0, 2 }

In [40]:
tile_data.get_data(cpu_tile_index);

In [41]:
Contingency<dim + 1> tile_contingency;
tile_contingency.allocate(tile_data.range_dofs);
Contingency<dim> tile_contingency_margins_1[dim + 1];
tile_contingency.allocate_margins(tile_contingency_margins_1);
Contingency<dim - 1> tile_contingency_margins_2[dim];
tile_contingency_margins_1[dim].allocate_margins(tile_contingency_margins_2);
double tile_IGs[dim];

unsigned int column_indices[dim] = {0, 1, 2};

tile_contingency.fill_with(1e-3);
for(unsigned int dim_index = 0; dim_index < dim + 1; ++dim_index) {
    tile_contingency_margins_1[dim_index].fill_with(0.);
}
for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
    tile_contingency_margins_2[dim_index].fill_with(0.);
}
tile_data.fill_in_contingency(tile_contingency, column_indices);

tile_contingency.compute_IGs(tile_IGs, tile_contingency_margins_1, tile_contingency_margins_2);
tile_IGs

{ 89.547, 82.8193, 97.6703 }

In [42]:
// #include <algorithm>
// std::max(2, 3)

In [43]:
template<unsigned int dim>
void compute_IGs(unsigned int (&column_indices)[dim],
                 TileData<dim> &tile_data,
                 Contingency<dim + 1> &tile_contingency,
                 Contingency<dim> (&tile_contingency_margins_1)[dim + 1],
                 Contingency<dim - 1> (&tile_contingency_margins_2)[dim],
                 double (&tile_IGs)[dim]) {
    //
    tile_contingency.fill_with(1e-3);
    for(unsigned int dim_index = 0; dim_index < dim + 1; ++dim_index) {
        tile_contingency_margins_1[dim_index].fill_with(0.);
    }
    for(unsigned int dim_index = 0; dim_index < dim; ++dim_index) {
        tile_contingency_margins_2[dim_index].fill_with(0.);
    }
    tile_data.fill_in_contingency(tile_contingency, column_indices);

    for (unsigned int dim_index = 0; dim_index < dim; ++dim_index)
        tile_IGs[dim_index] = 0;
    tile_contingency.compute_IGs(tile_IGs, tile_contingency_margins_1, tile_contingency_margins_2);

//     //
//     for (unsigned int dim_index = 0; dim_index < dim; ++dim_index)
//         std::cout << column_indices[dim_index] << " ";
//     std::cout << std::endl;
//     for (unsigned int dim_index = 0; dim_index < dim; ++dim_index)
//         std::cout << tile_IGs[dim_index] << " ";
//     std::cout << std::endl;
}

In [47]:
compute_IGs(column_indices,
            tile_data,
            tile_contingency,
            tile_contingency_margins_1,
            tile_contingency_margins_2,
            tile_IGs); 

0 1 2 
89.54695122 82.81926308 97.67028552 


In [1]:
template<unsigned int dim>
void dummy_function(unsigned int (&column_indices)[dim],
                    TileData<dim> &tile_data,
                    Contingency<dim + 1> &tile_contingency,
                    Contingency<dim> (&tile_contingency_margins_1)[dim + 1],
                    Contingency<dim - 1> (&tile_contingency_margins_2)[dim],
                    double (&tile_IGs)[dim]) {
    std::cout << "Not implemented." << std::endl;
}

void dummy_function<2>(unsigned int (&column_indices)[2],
                       TileData<dim> &tile_data,
                       Contingency<3> &tile_contingency,
                       Contingency<2> (&tile_contingency_margins_1)[3],
                       Contingency<1> (&tile_contingency_margins_2)[2],
                       double (&tile_IGs)[2]) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
     column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            //
            compute_IGs(column_indices,
                        tile_data,
                        tile_contingency,
                        tile_contingency_margins_1,
                        tile_contingency_margins_2,
                        tile_IGs); 
        }
    }
    //
}

void dummy_function<3>(unsigned int (&column_indices)[3],
                       TileData<3> &tile_data,
                       Contingency<4> &tile_contingency,
                       Contingency<3> (&tile_contingency_margins_1)[4],
                       Contingency<2> (&tile_contingency_margins_2)[3],
                       double (&tile_IGs)[3]) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
         column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            for (column_indices[2] = std::max(column_indices[1] + 1, tile_data.corner_columns_0[2]);
                 column_indices[2] < tile_data.corner_columns_1[2]; ++column_indices[2]) {
                //
                compute_IGs(column_indices,
                            tile_data,
                            tile_contingency,
                            tile_contingency_margins_1,
                            tile_contingency_margins_2,
                            tile_IGs);
            }
        }
    }
    //
}

void dummy_function<4>(unsigned int (&column_indices)[4],
                       TileData<4> &tile_data,
                       Contingency<5> &tile_contingency,
                       Contingency<4> (&tile_contingency_margins_1)[5],
                       Contingency<3> (&tile_contingency_margins_2)[4],
                       double (&tile_IGs)[4]) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
         column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            for (column_indices[2] = std::max(column_indices[1] + 1, tile_data.corner_columns_0[2]);
                 column_indices[2] < tile_data.corner_columns_1[2]; ++column_indices[2]) {
                for (column_indices[3] = std::max(column_indices[2] + 1, tile_data.corner_columns_0[3]);
                     column_indices[3] < tile_data.corner_columns_1[3]; ++column_indices[3]) {
                    //
                    compute_IGs(column_indices,
                                tile_data,
                                tile_contingency,
                                tile_contingency_margins_1,
                                tile_contingency_margins_2,
                                tile_IGs);
                }
            }
        }
    }
    //
}

void dummy_function<5>(unsigned int (&column_indices)[5],
                       TileData<5> &tile_data,
                       Contingency<6> &tile_contingency,
                       Contingency<5> (&tile_contingency_margins_1)[6],
                       Contingency<4> (&tile_contingency_margins_2)[5],
                       double (&tile_IGs)[5]) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
         column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            for (column_indices[2] = std::max(column_indices[1] + 1, tile_data.corner_columns_0[2]);
                 column_indices[2] < tile_data.corner_columns_1[2]; ++column_indices[2]) {
                for (column_indices[3] = std::max(column_indices[2] + 1, tile_data.corner_columns_0[3]);
                     column_indices[3] < tile_data.corner_columns_1[3]; ++column_indices[3]) {
                    for (column_indices[4] = std::max(column_indices[3] + 1, tile_data.corner_columns_0[4]);
                         column_indices[4] < tile_data.corner_columns_1[4]; ++column_indices[4]) {
                        //
                        compute_IGs(column_indices,
                                    tile_data,
                                    tile_contingency,
                                    tile_contingency_margins_1,
                                    tile_contingency_margins_2,
                                    tile_IGs);
                    }
                }
            }
        }
    }
    //
}

input_line_7:3:21: error: no template named 'TileData'
                    TileData<dim> &tile_data,
                    ^
input_line_7:4:21: error: no template named 'Contingency'
                    Contingency<dim + 1> &tile_contingency,
                    ^
input_line_7:5:21: error: no template named 'Contingency'
                    Contingency<dim> (&tile_contingency_margins_1)[dim + 1],
                    ^
input_line_7:6:21: error: no template named 'Contingency'
                    Contingency<dim - 1> (&tile_contingency_margins_2)[dim],
                    ^
input_line_7:8:10: error: no member named 'cout' in namespace 'std'
    std::cout << "Not implemented." << std::endl;
    ~~~~~^
input_line_7:11:24: error: no template named 'TileData'
                       TileData<dim> &tile_data,
                       ^
input_line_7:11:33: error: use of undeclared identifier 'dim'
                       TileData<dim> &tile_data,
                                ^
input_line_7:12:24:

Interpreter Error: 

In [49]:
// Contingency<dim + 1> tile_contingency;
// tile_contingency.allocate(tile_data.range_dofs);
// Contingency<dim> tile_contingency_margins_1[dim + 1];
// tile_contingency.allocate_margins(tile_contingency_margins_1);
// Contingency<dim - 1> tile_contingency_margins_2[dim];
// tile_contingency_margins_1[dim].allocate_margins(tile_contingency_margins_2);

// unsigned int column_indices[dim];
// double tile_IGs[dim];

if (dim == 2) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
     column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            //
            compute_IGs(column_indices,
                        tile_data,
                        tile_contingency,
                        tile_contingency_margins_1,
                        tile_contingency_margins_2,
                        tile_IGs); 
        }
    }
    //
}
else if (dim == 3) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
         column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            for (column_indices[2] = std::max(column_indices[1] + 1, tile_data.corner_columns_0[2]);
                 column_indices[2] < tile_data.corner_columns_1[2]; ++column_indices[2]) {
                //
                compute_IGs(column_indices,
                            tile_data,
                            tile_contingency,
                            tile_contingency_margins_1,
                            tile_contingency_margins_2,
                            tile_IGs);
            }
        }
    }
    //
}
else if (dim == 4) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
         column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            for (column_indices[2] = std::max(column_indices[1] + 1, tile_data.corner_columns_0[2]);
                 column_indices[2] < tile_data.corner_columns_1[2]; ++column_indices[2]) {
                for (column_indices[3] = std::max(column_indices[2] + 1, tile_data.corner_columns_0[3]);
                     column_indices[3] < tile_data.corner_columns_1[3]; ++column_indices[3]) {
                    //
                    compute_IGs(column_indices,
                                tile_data,
                                tile_contingency,
                                tile_contingency_margins_1,
                                tile_contingency_margins_2,
                                tile_IGs);
                }
            }
        }
    }
    //
}
else if (dim == 5) {
    //
    for (column_indices[0] = tile_data.corner_columns_0[0];
         column_indices[0] < tile_data.corner_columns_1[0]; ++column_indices[0]) {
        for (column_indices[1] = std::max(column_indices[0] + 1, tile_data.corner_columns_0[1]);
             column_indices[1] < tile_data.corner_columns_1[1]; ++column_indices[1]) {
            for (column_indices[2] = std::max(column_indices[1] + 1, tile_data.corner_columns_0[2]);
                 column_indices[2] < tile_data.corner_columns_1[2]; ++column_indices[2]) {
                for (column_indices[3] = std::max(column_indices[2] + 1, tile_data.corner_columns_0[3]);
                     column_indices[3] < tile_data.corner_columns_1[3]; ++column_indices[3]) {
                    for (column_indices[4] = std::max(column_indices[3] + 1, tile_data.corner_columns_0[4]);
                         column_indices[4] < tile_data.corner_columns_1[4]; ++column_indices[4]) {
                        //
                        compute_IGs(column_indices,
                                    tile_data,
                                    tile_contingency,
                                    tile_contingency_margins_1,
                                    tile_contingency_margins_2,
                                    tile_IGs);
                    }
                }
            }
        }
    }
    //
}
else {
    throw;
}

0 1 200 
69.18364254 66.15935688 78.68873284 
0 1 201 
69.53227324 64.23902736 77.95898715 
0 1 202 
68.87627499 64.3151615 79.13147833 
0 1 203 
81.6001161 71.68572663 89.52093413 
0 1 204 
79.74891664 79.70803604 88.52460374 
0 1 205 
82.95854201 76.85384704 90.61020476 
0 1 206 
75.36427624 71.21888737 86.31064037 
0 1 207 
71.25567844 78.63033227 85.65424827 
0 1 208 
79.09208182 64.63881221 85.61386426 
0 1 209 
74.6214828 80.39844762 87.80185875 
0 1 210 
81.19033759 71.69159259 90.28982124 
0 1 211 
70.96639076 66.86832357 88.58971476 
0 1 212 
73.74630938 71.55617681 85.55782397 
0 1 213 
67.28224558 77.28143987 89.05387535 
0 1 214 
89.62389926 70.95702216 94.53458722 
0 1 215 
70.15686272 68.85041849 91.56656486 
0 1 216 
75.7871351 76.08466879 87.06428492 
0 1 217 
66.53926153 76.3155148 85.62293499 
0 1 218 
66.58256286 83.15726636 78.16128898 
0 1 219 
75.49417166 75.33219419 94.30114356 
0 1 220 
64.41136078 65.17036382 84.33712064 
0 1 221 
68.88845857 78.96918125 85.059

66.53520892 67.21293421 74.51749502 
0 9 204 
65.63789113 72.43223864 70.71815965 
0 9 205 
81.25129053 78.04075263 81.26646365 
0 9 206 
69.90815813 70.77738376 75.33849006 
0 9 207 
73.56816657 72.12280489 68.61607419 
0 9 208 
84.7060146 77.5378879 87.98229325 
0 9 209 
72.12421155 81.67519124 78.54795567 
0 9 210 
71.92726606 75.205371 83.27295296 
0 9 211 
72.02003845 68.10302759 79.29377208 
0 9 212 
70.79157133 69.02600455 72.49700502 
0 9 213 
76.94447139 67.89821263 69.14000142 
0 9 214 
78.96678242 66.7480496 79.79496797 
0 9 215 
64.85168806 59.19070838 71.37620805 
0 9 216 
91.68667587 84.06933952 84.51830895 
0 9 217 
65.6607139 75.81548644 74.59225993 
0 9 218 
72.63633852 91.61209807 76.085474 
0 9 219 
64.913402 57.74833378 66.18663645 
0 9 220 
72.79668234 67.1920957 75.82820582 
0 9 221 
69.09652731 75.52917015 71.08879413 
0 10 200 
72.7094904 67.14785842 64.78393055 
0 10 201 
74.9768265 88.49133338 87.31798934 
0 10 202 
76.35692219 76.22056271 76.1435757 
0 10 203

0 17 204 
79.17674817 73.84310345 74.60986023 
0 17 205 
72.18895874 65.14333325 70.84988005 
0 17 206 
74.33826008 72.69486319 79.73680527 
0 17 207 
71.40301471 80.79401347 79.76811855 
0 17 208 
87.23772586 68.80981087 81.73505201 
0 17 209 
72.43860812 73.70941506 73.06301527 
0 17 210 
66.11293606 61.07034942 71.61876717 
0 17 211 
66.97286582 55.47529511 69.14687539 
0 17 212 
71.8764431 80.49048602 86.44232227 
0 17 213 
76.06516965 79.28710103 83.00972559 
0 17 214 
74.51254615 65.55749203 81.08524618 
0 17 215 
82.70438065 70.30083915 84.9671746 
0 17 216 
73.67930171 70.68803685 73.61784205 
0 17 217 
75.77173053 85.22633888 86.48394815 
0 17 218 
80.58094609 96.05634204 83.01055375 
0 17 219 
83.87344643 72.05844571 82.97758416 
0 17 220 
72.66383523 68.37982944 79.49677534 
0 17 221 
72.16180165 79.4675533 77.50801306 
0 18 200 
81.40120748 85.22209129 77.88605992 
0 18 201 
67.13742671 76.14608487 70.00063733 
0 18 202 
75.15254176 79.48648304 74.43739253 
0 18 203 
62.957

0 25 204 
79.00298945 70.09836583 67.21529188 
0 25 205 
84.48306397 94.64766598 96.70438205 
0 25 206 
61.6789659 60.62319937 64.01531072 
0 25 207 
65.9612626 67.97694768 63.30122203 
0 25 208 
85.63531337 70.0692419 79.3446523 
0 25 209 
91.82685021 85.59750367 81.30127315 
0 25 210 
70.64413145 70.11139554 77.00998255 
0 25 211 
69.99824588 68.61088409 78.63263363 
0 25 212 
71.98216435 75.06912706 77.37113258 
0 25 213 
65.7341805 71.65860724 71.73140107 
0 25 214 
79.87545541 68.2545849 80.13250831 
0 25 215 
72.31494785 60.45013941 71.46664413 
0 25 216 
81.10855062 74.88293437 74.16290884 
0 25 217 
76.30587101 75.78458688 73.39236542 
0 25 218 
78.13075235 84.61634331 67.92072428 
0 25 219 
77.86544895 74.12180346 81.39111118 
0 25 220 
72.54767987 68.7183537 76.18546887 
0 25 221 
81.71852766 90.11352348 84.5041525 
0 26 200 
70.19017625 69.85978013 73.35926889 
0 26 201 
83.29044692 73.26297395 77.95304655 
0 26 202 
50.61766131 52.46928466 58.25571429 
0 26 203 
73.50326499

0 33 204 
57.68774718 71.10811103 80.4334768 
0 33 205 
61.60134963 59.90389319 74.16904898 
0 33 206 
68.61511335 80.43009389 96.03064495 
0 33 207 
71.4042322 68.64941049 76.18212455 
0 33 208 
85.33010412 74.80123917 96.28508929 
0 33 209 
65.44848216 75.24843188 83.16064107 
0 33 210 
81.588528 71.20929125 90.31631797 
0 33 211 
60.56802009 71.32856391 93.55875317 
0 33 212 
67.77075426 67.13549232 81.64593755 
0 33 213 
61.59708096 76.05415028 88.33538383 
0 33 214 
67.41732192 69.50453147 93.5908946 
0 33 215 
67.82757389 60.14582661 83.37077104 
0 33 216 
70.99528617 62.71277622 74.20119041 
0 33 217 
67.55923122 77.31784558 87.13406384 
0 33 218 
70.86475832 85.86835992 81.3811806 
0 33 219 
77.86524211 65.41062531 84.88837275 
0 33 220 
70.26978161 69.158201 88.83375588 
0 33 221 
79.29851323 78.96093867 85.56000741 
0 34 200 
66.73149137 66.51619333 66.06630409 
0 34 201 
62.51491002 67.93820873 68.67890332 
0 34 202 
65.77983253 72.05780662 73.89485824 
0 34 203 
68.66254422

...

91 93 219 
69.61807123 66.61972097 66.603456 
91 93 220 
69.46821744 69.81109589 69.0827622 
91 93 221 
81.69545172 75.42631261 76.44549945 
91 94 200 
69.89176103 70.65753781 78.65934691 
91 94 201 
54.93016652 63.29426762 70.715814 
91 94 202 
66.56919712 69.92818086 79.10826495 
91 94 203 
77.42471048 68.00455003 88.58422841 
91 94 204 
72.89876849 77.60027005 84.93984545 
91 94 205 
76.07201867 78.802013 84.6116338 
91 94 206 
60.60117974 63.8729425 74.23290431 
91 94 207 
69.68690677 70.04555804 75.20943765 
91 94 208 
84.41212371 72.19472005 83.32006134 
91 94 209 
68.53062504 68.87688874 76.67058463 
91 94 210 
71.47210663 60.78546388 75.45729572 
91 94 211 
68.63920467 65.07530917 72.10795418 
91 94 212 
74.06536486 72.42610624 80.18436304 
91 94 213 
58.34965683 58.39879101 72.27462715 
91 94 214 
75.79503904 75.74565859 79.37960816 
91 94 215 
82.05193235 66.47903731 89.28017636 
91 94 216 
72.04201256 60.95307739 77.35911816 
91 94 217 
55.487309 61.12876318 74.21476357 
91 

92 94 215 
85.50925604 74.56243288 92.01143734 
92 94 216 
75.34831691 74.26683109 79.93935981 
92 94 217 
54.47715008 67.08997693 72.47854195 
92 94 218 
79.72656659 74.44049669 80.32834105 
92 94 219 
82.32818278 78.46032691 86.67784639 
92 94 220 
82.13251682 78.13458836 83.11502768 
92 94 221 
70.96591909 60.1391973 72.72557187 
92 95 200 
75.78329989 74.52755237 77.76537473 
92 95 201 
67.19150705 63.40558691 73.83647603 
92 95 202 
80.22933269 79.2422314 85.72332276 
92 95 203 
69.87965267 72.70626272 79.01697761 
92 95 204 
68.64409313 59.49901571 80.04586316 
92 95 205 
79.80311057 74.96759843 88.9245458 
92 95 206 
63.95695571 65.81734436 72.30980431 
92 95 207 
72.48415311 74.04994611 83.74980304 
92 95 208 
77.94885091 63.03271634 83.55057084 
92 95 209 
64.96873642 71.09888494 79.04087647 
92 95 210 
68.53628484 61.81881989 72.33453132 
92 95 211 
70.34646442 70.8014023 79.03304833 
92 95 212 
67.65713721 64.74293294 68.8537225 
92 95 213 
72.64243875 70.90768103 82.9953303

93 96 211 
60.29819167 67.63383922 73.8746445 
93 96 212 
84.40886004 83.75535432 77.04165608 
93 96 213 
69.58649564 76.16102899 76.35970881 
93 96 214 
79.98484233 70.81770143 73.60988756 
93 96 215 
65.03958206 76.4702167 75.15590436 
93 96 216 
85.78153968 94.5727662 81.494482 
93 96 217 
63.03168836 81.64075947 76.793798 
93 96 218 
86.59624476 85.9265382 81.51055272 
93 96 219 
80.26776856 80.20717296 83.04339698 
93 96 220 
87.95817052 79.52597752 84.99136154 
93 96 221 
71.25466219 75.0299043 75.63079127 
93 97 200 
79.21537891 86.16329901 78.90276061 
93 97 201 
77.7689911 69.30438135 73.68929015 
93 97 202 
57.98462526 67.29645662 57.59681957 
93 97 203 
70.07137689 76.5664627 67.72751882 
93 97 204 
75.8069709 77.4541934 70.9781966 
93 97 205 
72.23773214 71.50040704 66.00434163 
93 97 206 
77.50928878 76.68400331 77.63550144 
93 97 207 
75.48829592 78.40820831 66.65125129 
93 97 208 
82.43925329 88.15401273 76.88185689 
93 97 209 
67.96086835 64.40341098 58.26928238 
93 97 

94 99 207 
78.16463878 81.7919539 80.43072715 
94 99 208 
71.35214119 77.80878912 69.83296913 
94 99 209 
63.63297663 69.01085635 70.26705832 
94 99 210 
83.88317263 81.33104895 78.43248042 
94 99 211 
65.0203831 77.17235971 73.75735159 
94 99 212 
70.63028218 80.52302379 79.75826434 
94 99 213 
75.5072482 70.944006 77.98521869 
94 99 214 
65.80343402 89.68261626 86.38342775 
94 99 215 
67.36783613 77.44108639 77.78557277 
94 99 216 
74.1830497 72.61949632 71.0528443 
94 99 217 
69.14820198 61.95238234 73.79607929 
94 99 218 
73.63669824 83.30852364 77.75260318 
94 99 219 
71.8836383 73.06052154 71.25249022 
94 99 220 
69.68337089 83.62456336 78.44937929 
94 99 221 
74.71188649 79.39792589 74.99988374 
95 96 200 
67.70350742 76.77880213 70.10054297 
95 96 201 
72.33759278 69.69546591 67.6801009 
95 96 202 
79.84462649 74.77500414 71.60866023 
95 96 203 
74.13520447 81.35179283 81.82878379 
95 96 204 
71.02374697 69.60396986 72.3454059 
95 96 205 
81.77162299 85.69894272 86.16004396 
95

97 98 203 
62.73640246 61.45045049 69.44852364 
97 98 204 
79.01387219 67.09330828 72.6064652 
97 98 205 
77.84748471 80.65859818 84.7671389 
97 98 206 
76.79575964 70.78604476 81.25418864 
97 98 207 
84.18576436 77.01472336 78.51960995 
97 98 208 
72.15674091 71.3208056 76.10534042 
97 98 209 
82.53022736 79.86709771 80.51744296 
97 98 210 
88.45257138 87.17369435 83.99083736 
97 98 211 
71.62566598 68.52964514 71.80270841 
97 98 212 
81.36151217 67.62346149 71.8046085 
97 98 213 
70.24589294 70.46535849 75.93008214 
97 98 214 
88.05341311 76.96218515 84.71234048 
97 98 215 
68.46696928 67.9354308 76.84174673 
97 98 216 
65.57815654 78.80976014 75.83282969 
97 98 217 
79.57295953 75.97515809 79.2645982 
97 98 218 
82.31407782 70.09181849 71.31489274 
97 98 219 
75.79762782 61.06897164 70.63377363 
97 98 220 
76.6674905 76.85572703 83.22770836 
97 98 221 
69.30226051 73.5289672 78.11849035 
97 99 200 
72.99410168 68.52802713 78.91600239 
97 99 201 
65.9978706 69.09612622 75.71701882 
9

In [51]:
cpu_tile_indices.reset();
while(!cpu_tile_indices.is_exhausted()) {
    
    cpu_tile_indices.next()
}

{ 0, 0, 2 }